In [ ]:
import numpy as np
import xarray 
import rioxarray
import holoviews as hv
from holoviews import opts
import geoviews as gv
import datashader as ds
import cartopy.crs as ccrs
import hvplot
from holoviews.operation.datashader import regrid, shade
from bokeh.tile_providers import STAMEN_TONER
import rasterio
from osgeo import gdal
from pathlib import Path
import panel
import hvplot.xarray
from ipywidgets import interact, Dropdown, FloatSlider, IntSlider, SelectMultiple, Text
import matplotlib.pyplot as plt
import xmltodict
import zipfile

hv.extension('bokeh', width=1000)

# DESIS L2

## Extract GeoTIFF from zip package

In [ ]:
filename = '../../../data/DESIS/dims_op_oc_dfd2_693797361_1/DESIS.HSI.L2A/p1/DESIS-HSI-L2A-DT0581213156_014-20210427T051649-V0220.zip'
desisfile = zipfile.ZipFile(filename, 'r')

In [ ]:
desisfile.filelist[1].filename

In [ ]:
desisfile.filelist[3].filename

In [ ]:
desistif = desisfile.read('DESIS-HSI-L2A-DT0581213156_014-20210427T051649-V0220/DESIS-HSI-L2A-DT0581213156_014-20210427T051649-V0220-SPECTRAL_IMAGE.tif')
desismetadata = desisfile.read('DESIS-HSI-L2A-DT0581213156_014-20210427T051649-V0220/DESIS-HSI-L2A-DT0581213156_014-20210427T051649-V0220-SPECTRAL_IMAGE.tif')

In [ ]:
filename = desisfile.extract('DESIS-HSI-L2A-DT0581213156_014-20210427T051649-V0220/DESIS-HSI-L2A-DT0581213156_014-20210427T051649-V0220-SPECTRAL_IMAGE.tif')

In [ ]:
filenamexml = desisfile.extract('DESIS-HSI-L2A-DT0581213156_014-20210427T051649-V0220/DESIS-HSI-L2A-DT0581213156_014-20210427T051649-V0220-METADATA.xml')

In [ ]:
fileptr = open(filenamexml,"r")

In [ ]:
xml_content= fileptr.read()

In [ ]:
desisdict = xmltodict.parse(xml_content)

In [ ]:
#desisdict

## Create useful band names and apply to GeoTIFF

In [ ]:
bands = []
for item in desisdict['hsi_doc']['specific']['bandCharacterisation']['band']:
    #print(item['bandNumber'], item['wavelengthCenterOfBand'])#, item['response'], item['wavelengths'], item['gainOfBand'], item['offsetOfBand'])
    bandnumber = (int(item['bandNumber']))
    bands.append((bandnumber, 'VNIR'+str(bandnumber)+' '+str(int(item['wavelengthCenterOfBand'].split('.')[0]))))

In [ ]:
def set_band_descriptions(filepath, bands):
    """
    filepath: path/virtual path/uri to raster
    bands:    ((band, description), (band, description),...)
    """
    ds = gdal.Open(filepath, gdal.GA_Update)
    for band, desc in bands:
        rb = ds.GetRasterBand(band)
        rb.SetDescription(desc)
    del ds

In [ ]:
# Update the GeoTIFF band names to enable easier correspondence with other bands - use cwl for bands from metadata
set_band_descriptions(filename, bands)

In [ ]:
#gdal.Info(filename).split('\n')

In [ ]:
geofilename = filename.replace('.tif', '_geo.tif')

## Warp GeoTIFF to lat/lon

In [ ]:
gdal.Warp(geofilename, filename, options="-t_srs EPSG:4326 -overwrite -tr 0.0003 0.0003 -tap")

In [ ]:
desisxarray = xarray.open_dataset(filename)

In [ ]:
desisxarraygeo = xarray.open_dataset(geofilename)

In [ ]:
#gdal.Info(filename+'_geo.tif').split('\n')

In [ ]:
(desisxarray.band_data[[1,45,95]]/5).hvplot.rgb( geo=True, x='x', y='y', bands='band', data_aspect=1, flip_yaxis=False, xaxis=False, yaxis=None, title="RGB Plot with HVPlot", width=1200)

In [ ]:
(desisxarraygeo.band_data[[1,45,95]]/5).hvplot.rgb( geo=True, x='x', y='y', bands='band',  title="RGB Plot with HVPlot", width=1200, tiles='ESRI', alpha=0.5)

## Make an interactive RGB plot

In [ ]:
redW = Dropdown(options = list(desisxarray.band.values))#prismaxarray.band.values.tolist())
greenW = Dropdown(options = list(desisxarray.band.values))#prismaxarray.band.values.tolist())
blueW = Dropdown(options = list(desisxarray.band.values))#prismaxarray.band.values.tolist())
startW = Dropdown(options = ['Pause', 'Go'])

@interact(red = redW, green = greenW, blue = blueW, start = startW)

def rgb_combo(red, green, blue, start):
    redW.options = list(desisxarray.band.values)#prismaxarray.band.values.tolist()
    greenW.options = list(desisxarray.band.values)#prismaxarray.band.values.tolist()
    blueW.options = list(desisxarray.band.values)#prismaxarray.band.values.tolist()
    if start == 'Go' and red != green and green !=blue and red != blue:
        #(prismaxarray[[red,green,blue]]/100).to_array().hvplot.rgb(x='x', y='y', bands='variable', data_aspect=1, flip_yaxis=False, xaxis=False, yaxis=None)
        desisxarray1000.isel(band=[red,green,blue]).hvplot.rgb(x='x', y='y', bands='band', data_aspect=1, flip_yaxis=False, xaxis=False, yaxis=None, title="RGB Plot with HVPlot", width=1200)
    return(red,green,blue,start)

In [ ]:
(desisxarray.isel(band=[78,150,210])/50).hvplot.rgb(x='x', y='y', bands='band', data_aspect=1, flip_yaxis=False, xaxis=False, yaxis=None, title="RGB Plot with HVPlot", width=1200)

In [ ]:
# TODO - Make the RGB plot interactive. It is possible as below but doesn't work with iwidget method above
(desisxarray[['SWIR74 1687','SWIR18 1088','VNIR28 618']]/100).to_array().hvplot.rgb(x='x', y='y', bands='variable', data_aspect=1, flip_yaxis=False, xaxis=False, yaxis=None, title="RGB Plot with HVPlot", width=1200)

In [ ]:
desisxarray